In [1]:
# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [2]:
import numpy as np

class SimilarityClustering:
    def __init__(self, threshold):
        self.threshold = threshold
        self.labels_ = None

    def fit(self, features):
        cosine_similarities = np.array([cosine_similarity(features[i], features[i + 1])
                                        for i in range(len(features) - 1)])

        cosine_similarities = np.insert(cosine_similarities, 0, cosine_similarity(features[0], features[0]))

        self.labels_ = label_clusters(cosine_similarities, self.threshold)

cosine_similarity = lambda vec1, vec2: np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def label_clusters(cosine_similarities, threshold):
    clusters = np.zeros(len(cosine_similarities), dtype=int)
    current_cluster = 0
    for i in range(1, len(cosine_similarities)):
        #print(i,cosine_similarities[i])

        if cosine_similarities[i] < threshold:
            current_cluster += 1
        clusters[i] = current_cluster
    #print(clusters)
    return clusters

def clustering_function(model):
    def get_labels(features):
        model.fit(features)
        labels = model.labels_
        return labels
    return get_labels


In [3]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from models.resnet18 import get_model as get_resnet18
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from clustering.model import clustering_function
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')


if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset = load_dataset(VALIDATION_DATASET.keys())

EXPERIMENT_DEFINITIONS = [
   {
      "name": "ResnetAglomerative",
      "model": get_resnet18(device),
      "preprocessing": BASELINE,
      "dataset": dataset,
      "grouper_function": clustering_function(AgglomerativeClustering(None, distance_threshold=50)),
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   },
      {
      "name": "SimilarityClustering",
      "model": get_resnet18(device),
      "preprocessing": BASELINE,
      "dataset": dataset,
      "grouper_function": clustering_function(SimilarityClustering(threshold=0.999)),
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   },
]
for exp_definition in EXPERIMENT_DEFINITIONS:
   experiment(**exp_definition)


Experiment name: ResnetAglomerative video.mp4
Precision - Mean: 0.70, std: 0.0302777997
Recall - Mean: 0.64, std: 0.0435561371
F1-Score - Mean: 0.62, std: 0.0372928944
Accuracy - Mean: 0.64, std: 0.0435561371

Experiment name: ResnetAglomerative video2.mp4
Precision - Mean: 0.70, std: 0.0381381662
Recall - Mean: 0.72, std: 0.0433557936
F1-Score - Mean: 0.69, std: 0.0422632349
Accuracy - Mean: 0.72, std: 0.0433557936

Experiment name: ResnetAglomerative AVG
Precision - Mean: 0.70, std: 0.0004677915
Recall - Mean: 0.68, std: 0.0398856640
F1-Score - Mean: 0.66, std: 0.0329354093
Accuracy - Mean: 0.68, std: 0.0398856640

Experiment name: SimilarityClustering video.mp4
Precision - Mean: 0.75, std: 0.0457403567
Recall - Mean: 0.38, std: 0.0299167024
F1-Score - Mean: 0.45, std: 0.0287341270
Accuracy - Mean: 0.38, std: 0.0299167024

Experiment name: SimilarityClustering video2.mp4
Precision - Mean: 0.86, std: 0.0233328160
Recall - Mean: 0.45, std: 0.0228671944
F1-Score - Mean: 0.56, std: 0.02

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.6 MB

In [6]:
from ui import get_tagging_ui


tag_ui = get_tagging_ui()
tag_ui.launch()

ImportError: ignored